In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim

import numpy as np

In [2]:
# UNIPoint model
class UNIPoint(nn.Module):
    def __init__(self, batch_size, n_inputs, n_neurons, n_parameters, n_basis_functions):
      """
      Input parameters:
      n_neurons - number of neurons inside RNN
      n_parameters - expecteed number of parameters in basis function
      n_basis_functions - number of basis functions
      """
      super(UNIPoint, self).__init__()
        
      self.rnn = nn.RNNCell(n_inputs, n_neurons)
      self.hx = torch.randn(batch_size, n_neurons) # initialize hidden state 
      self.h2p = nn.Linear(n_neurons, n_parameters)
      self.parameters = torch.randn(batch_size, n_parameters) # initialize parameters for intensity functions
      self.basis_res1 = torch.randn(batch_size, n_parameters)
      self.basis_res2 = torch.randn(batch_size, n_parameters)
      self.Softplus = torch.nn.Softplus(beta = 1)
      self.result = torch.randn(1)

    def ReLU(self, parameters, time):
      """Function to apply Rectified Linear Unit (ReLU) as basis function inside network 
        Input parameters:
          parameters - alpha, beta for basis function's value calculation
          time - column-vector with time which had been spent since the begining of 
                  temporal point process (TPP)
      """
        
      self.output = torch.relu(self.parameters[:,0] @ time + self.parameters[:,1]) 
      return self.output
    
    def PowerLaw(self, parameters, time):
      """Function to apply Power Law (PL) as basis function inside network 
        Input parameters:
          parameters - alpha, beta for basis function's value calculation
          time - column-vector with time which had been spent since the begining of 
                  temporal point process (TPP)
      """
      self.output = self.parameters[:,0] * (1 + time) ** (- self.parameters[:,1])
      return self.output


    def forward(self, X, time):
      """Input parameters:
          X - batch with data
          time - column-vector with time which had been spent since the begining of 
                  temporal point process (TPP)
      """
        
      output = []

      # for each time step
      for i in range(2):
          self.hx = self.rnn(X[i], self.hx)
          self.parameters = self.h2p(self.hx)
          self.basis_res1 = self.ReLU(self.parameters, time)
          self.basis_res2 = self.PowerLaw(self.parameters, time)
          self.result = self.Softplus(self.basis_res1 + self.basis_res2)

          output.append(self.hx)

        
      return output, self.hx, self.parameters, self.basis_res1, self.basis_res2, self.result

In [3]:
# model evaluation

FIXED_BATCH_SIZE = 4 # our batch size is fixed for now
N_INPUT = 3
N_NEURONS = 1

N_PARAMETERS = 2
N_BASIS_FUNCTIONS = 8

# for now we generate random values for time in [0,T]
TIME = torch.from_numpy(np.sort(np.random.sample(FIXED_BATCH_SIZE))).type(torch.float) # TIME size is (FIXED_BATCH_SIZE, 1)

X_batch = torch.tensor([[[0,1,2], [3,4,5], 
                         [6,7,8], [9,0,1]],
                        [[9,8,7], [0,0,0], 
                         [6,5,4], [3,2,1]]], dtype = torch.float) # X0 and X1


model = UNIPoint(FIXED_BATCH_SIZE, N_INPUT, N_NEURONS, N_PARAMETERS, N_BASIS_FUNCTIONS)
print(model)
output_val, states_val, params_value, basis_value1, basis_value2, res = model(X_batch, TIME)
print()
print('output_val')
print(output_val) # contains all output for all timesteps
print()
print('states_val')
print(states_val) # contains values for final state or final timestep, i.e., t=1
print()
print('params_value')
print(params_value) # contains values for parameters for final timesteps
print()
print('basis_value1')
print(basis_value1) # contains values of basis functions (now it is ReLU)
print()
print('basis_value2')
print(basis_value2) # contains values of basis functions (now it is Power Law)
print()
print('res')
print(res) # contains values of SoftPlus activation

UNIPoint(
  (rnn): RNNCell(3, 1)
  (h2p): Linear(in_features=1, out_features=2, bias=True)
  (Softplus): Softplus(beta=1, threshold=20)
)

output_val
[tensor([[-0.8374],
        [-0.9988],
        [-1.0000],
        [ 0.8818]], grad_fn=<TanhBackward>), tensor([[-1.0000],
        [-0.0606],
        [-0.9990],
        [-0.8425]], grad_fn=<TanhBackward>)]

states_val
tensor([[-1.0000],
        [-0.0606],
        [-0.9990],
        [-0.8425]], grad_fn=<TanhBackward>)

params_value
tensor([[0.8359, 1.0991],
        [0.8950, 0.1929],
        [0.8360, 1.0982],
        [0.8458, 0.9471]], grad_fn=<AddmmBackward>)

basis_value1
tensor([2.7156, 1.8094, 2.7147, 2.5637], grad_fn=<ReluBackward0>)

basis_value2
tensor([0.6745, 0.8342, 0.4953, 0.5329], grad_fn=<MulBackward0>)

res
tensor([3.4233, 2.7123, 3.2496, 3.1408], grad_fn=<SoftplusBackward>)
